In [2]:
import gspread
from google.oauth2.service_account import Credentials
import pandas as pd

# 認証情報の設定
SERVICE_ACCOUNT_FILE = 'C:\\Users\\udano\\OneDrive\\デスクトップ\\Tech0\\Step3-1\\SUUMO\\googlespreadsheet.json'
SCOPES = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']
creds = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
gc = gspread.authorize(creds)

# 対象のスプレッドシートとワークシートを指定
SPREADSHEET_KEY = '1-l-E6gb82lZQKlDSMpQX1LhH_z8bLG-u591WAzJv8p4'
sheet1 = gc.open_by_key(SPREADSHEET_KEY).worksheet("シート1")
sheet2 = gc.open_by_key(SPREADSHEET_KEY).worksheet("シート2")
sheet3 = gc.open_by_key(SPREADSHEET_KEY).worksheet("シート3")

# Google スプレッドシートからデータを読み込む
data_sheet1 = sheet1.get_all_values()
data_sheet2 = sheet2.get_all_values()

# データフレームに変換
df_sheet1 = pd.DataFrame(data_sheet1[1:], columns=data_sheet1[0])
df_sheet2 = pd.DataFrame(data_sheet2[1:], columns=data_sheet2[0])

# データの整形（共通の列を保持）
common_columns = list(set(df_sheet1.columns) & set(df_sheet2.columns))
df_sheet1 = df_sheet1[common_columns]
df_sheet2 = df_sheet2[common_columns]

# データをマージする
merged_df = pd.concat([df_sheet1, df_sheet2])

# 'address' 列から区の名前を抽出して 'ward' 列を作成する関数
def extract_ward(address):
    if '東京都' in address and '区' in address:
        return address.split('東京都')[1].split('区')[0] + '区'
    else:
        return ''  # 東京都や区が含まれていない場合は空文字を返す
# apply関数を使用して、各アドレスに対してextract_ward関数を適用
merged_df['ward'] = merged_df['address'].apply(extract_ward)

# 列の順序を指定
column_order = ['title', 'address', 'ward','age', 'floor', 'fee', 'management_fee', 'deposit', 'gratuity', 'floor_plan', 'area', 'access1_route', 'access1_nearest_station', 'access1_walking_minutes', 'access2_route', 'access2_nearest_station', 'access2_walking_minutes']
merged_df = merged_df[column_order]

# 重複を削除（全ての重複を削除）
merged_df.drop_duplicates(subset=['title', 'floor', 'fee', 'floor_plan', 'area'], keep=False, inplace=True)

# NaN値を空文字列に置き換える
merged_df.fillna('', inplace=True)

# 重複削除したデータ数をカウント
duplicates_count = len(merged_df)
print("重複削除が", duplicates_count, "件行われました.")

# データフレームをリスト形式に変換（列名を含む）
data_to_export = [merged_df.columns.tolist()] + merged_df.values.tolist()

# スプレッドシートのデータをクリアし、新たにデータを書き込む
sheet3.clear()  # 既存のデータをクリア
sheet3.update('A1', data_to_export)  # A1セルからデータを書き込む


重複削除が 38719 件行われました.


C:\Users\udano\AppData\Local\Temp\ipykernel_1940\375235038.py:61: DeprecationWarning: [Deprecated][in version 6.0.0]: Method signature's arguments 'range_name' and 'values' will change their order. We recommend using named arguments for minimal impact. In addition, the argument 'values' will be mandatory of type: 'List[List]'. (ex) Worksheet.update(values = [[]], range_name=) 
  sheet3.update('A1', data_to_export)  # A1セルからデータを書き込む


{'spreadsheetId': '1-l-E6gb82lZQKlDSMpQX1LhH_z8bLG-u591WAzJv8p4',
 'updatedRange': "'シート3'!A1:Q38720",
 'updatedRows': 38720,
 'updatedColumns': 17,
 'updatedCells': 658240}

In [4]:
import gspread
from google.oauth2.service_account import Credentials
import pandas as pd
import sqlite3

# Googleスプレッドシートからデータを取得する関数
def get_data_from_google_sheet():
    # 認証情報の設定
    SERVICE_ACCOUNT_FILE = 'C:\\Users\\udano\\OneDrive\\デスクトップ\\Tech0\\Step3-1\\SUUMO\\googlespreadsheet.json'
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']
    creds = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
    gc = gspread.authorize(creds)

    # 対象のスプレッドシートとワークシートを指定
    SPREADSHEET_KEY = '1-l-E6gb82lZQKlDSMpQX1LhH_z8bLG-u591WAzJv8p4'
    sheet = gc.open_by_key(SPREADSHEET_KEY).worksheet("シート3")

    # Google スプレッドシートからデータを読み込む
    data = sheet.get_all_values()

    # データフレームに変換
    df = pd.DataFrame(data[1:], columns=data[0])

    return df

# SQLiteデータベースにデータを書き込む関数
def write_data_to_sqlite(dataframe, database_path):
    conn = sqlite3.connect(database_path)
    cursor = conn.cursor()

    # データベースにデータを書き込む（データフレームをSQLiteテーブルとして作成）
    dataframe.to_sql('property_table', conn, if_exists='replace', index=False)

    # データベース接続を閉じる
    conn.close()

# メインの処理
if __name__ == "__main__":
    # Googleスプレッドシートからデータを取得
    data_df = get_data_from_google_sheet()

    # SQLiteデータベースにデータを書き込む
    database_path = 'property.db'  # SQLiteデータベースファイルのパスを指定
    write_data_to_sqlite(data_df, database_path)


In [6]:
import sqlite3
# SQLiteデータベースに接続
conn = sqlite3.connect('property.db')
# カーソルオブジェクトの作成
cursor = conn.cursor()
# テーブルから上位5行を選択
cursor.execute('SELECT * FROM property_table LIMIT 5')
# 結果を取得して表示
rows = cursor.fetchall()
for row in rows:
    print(row)
# 接続を閉じる
conn.close()

('奥山方', '東京都荒川区西日暮里２', '荒川区', '41', '1', '150000', '0', '150000', '150000', '4K', '73.75', 'ＪＲ山手線', '日暮里駅', '5', 'ＪＲ山手線', '西日暮里駅', '10')
('都営大江戸線 東中野駅 3階建 築21年', '東京都新宿区北新宿３', '新宿区', '21', '1', '190000', '10000', '0', '190000', '4SK', '80.0', '都営大江戸線', '東中野駅', '9', '東京メトロ東西線', '落合駅', '15')
('上中里2丁目戸建', '東京都北区上中里２', '北区', '47', '1-3', '135000', '0', '135000', '135000', '4K', '60.0', 'ＪＲ高崎線', '尾久駅', '5', 'ＪＲ京浜東北線', '上中里駅', '6')
('SORA赤塚新町', '東京都板橋区赤塚新町１', '板橋区', '63', '1-2', '130000', '0', '130000', '130000', '5K', '68.0', '東武東上線', '下赤塚駅', '6', '東京メトロ有楽町線', '地下鉄赤塚駅', '7')
('レジディア秋葉原', '東京都台東区上野５', '台東区', '17', '11', '133000', '10000', '133000', '133000', '2K', '30.56', 'ＪＲ山手線', '秋葉原駅', '8', '東京メトロ銀座線', '末広町駅', '4')
